In [7]:
import json 
import importlib 
import preprocess_utils 
importlib.reload(preprocess_utils)
from preprocess_utils import MultiTurnDataset

from transformers import AutoTokenizer

from datasets import Dataset
import datasets 

In [5]:
tokenizer = AutoTokenizer.from_pretrained("/data/dataset/huggingface/hub/models--THUDM--chatglm3-6b/snapshots/456aa875cf1f46623006edaa23103774ea9c0eae", trust_remote_code=True)

In [6]:
with open('formatted_data/tool_alpaca.jsonl', "r", encoding="utf-8") as f:
    if 'formatted_data/tool_alpaca.jsonl'.endswith(".json"):
        train_data = json.load(f)
    elif 'formatted_data/tool_alpaca.jsonl'.endswith(".jsonl"):
        train_data = [json.loads(line) for line in f]

In [8]:
train_dataset = MultiTurnDataset(
    train_data,
    tokenizer,
    2048,
)

In [38]:
def generater():
    num = len(train_dataset)
    for i in range(num):
        yield {"conversations": json.dumps(train_dataset[i])} 

In [40]:
new = Dataset.from_generator(generater)

Generating train split: 0 examples [00:00, ? examples/s]

In [45]:
new.save_to_disk('formatted_data/hf_dataset/tool_alpaca.hf')

Saving the dataset (0/1 shards):   0%|          | 0/4048 [00:00<?, ? examples/s]

In [48]:
new2 = datasets.load_from_disk('formatted_data/hf_dataset/tool_alpaca.hf')

In [50]:
new2[0] == new[0]

True

In [52]:
json.loads(new2[10]['conversations']) == train_dataset[10]

True

In [54]:
from typing import TYPE_CHECKING, Any, Dict, Generator, List, Literal, Tuple, Union
import json 

In [68]:
def preprocess_tokenized_dataset(examples: Dict[str, List[str]]) -> Dict[str, List[List[int]]]:
    model_inputs = {'input_ids': [], 'labels': []}
    for item in examples['conversations']:
        data = json.loads(item)
        model_inputs['input_ids'].append(data['input_ids'])
        model_inputs['labels'].append(data['labels'])
    return model_inputs

In [69]:
ss = new2.map(
    preprocess_tokenized_dataset,
    batched=True,
    remove_columns=['conversations'],
)

Map:   0%|          | 0/4048 [00:00<?, ? examples/s]

In [70]:
ss[0] == train_dataset[0]

True